    Create the environment:

In [2]:
""" Dan Lanier AE598RL """
import numpy as np
import gym

class grid_env(gym.Env):
    
    def __init__(self, p=None, rows=5, cols=5):
        if not (p is not None and isinstance(p, (np.ndarray, np.generic) ) and p.size == 2):
            p = np.array([0,0])
            
        self.rows = np.cumsum(np.int_(np.ones(rows))) - 1
        self.cols = np.cumsum(np.int_(np.ones(cols))) - 1
        self.rewards_dict = {'A': {'p':[0, 1], 'reward': 10, 'new_p':[4, 1]} , 
                             'B': {'p':[0, 3], 'reward': 5, 'new_p':[4, 3]}}
        self.p = self.set_p(p)
        self.done = False
        
    def step(self, dp):
        reward = self.get_reward_and_set_position(dp)
        
        return reward, self.get_p(), self.done, {}
    
    def get_reward_and_set_position(self, dp):
        maybe_p = np.add(self.p, dp)
        reward = 0
        
        if self.out_of_bounds(maybe_p):
            reward = -1
            
        elif all(maybe_p == self.rewards_dict['A']['p']):
            reward = self.rewards_dict['A']['reward']
            self.p = self.set_p(self.rewards_dict['A']['new_p'])
        elif all(maybe_p == self.rewards_dict['B']['p']):
            reward = self.rewards_dict['B']['reward']
            self.p = self.set_p(self.rewards_dict['B']['new_p'])
        else:
            self.p = self.set_p(maybe_p)
            
        return reward
            
    def out_of_bounds(self, p):
        bdck = self.check_bounds(p)
        if bdck['rows']['low'] or  bdck['rows']['high'] or bdck['cols']['low'] or bdck ['cols']['high']:
            out_of_bounds = True
        else:
            out_of_bounds = False
        return out_of_bounds
    
    def check_bounds(self, p):
        return {'rows': {'low': p[0] < self.rows.min(), 'high': p[0] > self.rows.max()}, \
                       'cols': {'low': p[1] < self.cols.min(), 'high': p[1] > self.cols.max()}}
        
    def get_p(self):
        return self.p
    
    def set_p(self, p):
        bounds_check_dict = self.check_bounds(p)
        if bounds_check_dict['rows']['low']:
            p[0] = self.rows.min()
        elif bounds_check_dict['rows']['high']:
            p[0] = self.rows.max()
        if bounds_check_dict['cols']['low']:
            p[1] = self.cols.min()
        elif bounds_check_dict['cols']['high']:
            p[1] = self.cols.max()
            
        return p
    
def get_random_move():
    return np.array([np.random.randint(0,3) -1, np.random.randint(0,3) -1])

    The next cell is the chaos agent

In [3]:
env = grid_env(np.array([0,0]))
print('Environment Initial Position:',env.p)

positives_reward_sum = 0
neg_rewards_sum = 0
negatives_limit = 10000
crash_limit = 1000
n_while = 0
landed_matrix = np.int_(np.zeros([5,5]))
Value_matrix = np.int_(np.zeros([5,5]))
while np.abs(neg_rewards_sum) < negatives_limit and n_while < crash_limit:
    n_while += 1
    maybe_p = get_random_move()
    reward, p, done, _ = env.step(maybe_p)
    
    if reward < 0:
        neg_rewards_sum += reward
        Value_matrix[p[0], p[1]] += reward
        landed_matrix[p[0], p[1]] += 1
    else:
        positives_reward_sum += reward
        if reward == 5:
            Value_matrix[0, 3] += reward
            landed_matrix[0, 3] += 1
        elif reward == 10:
            Value_matrix[0, 1] += reward
            landed_matrix[0, 1] += 1
        else:
            landed_matrix[p[0], p[1]] += 1

print('\nnumber of landings \n',landed_matrix)
print('\n')
print('Rewards - Value_matrix\n',Value_matrix)
print('\nnegative rewards',neg_rewards_sum,'\npositive rewards',positives_reward_sum)

Environment Initial Position: [0 0]

number of landings 
 [[21 12  6  8  1]
 [21 25 24 13  5]
 [33 35 30 40 24]
 [61 54 71 56 67]
 [51 90 88 91 73]]


Rewards - Value_matrix
 [[ -8 120  -2  40   0]
 [ -5   0   0   0   0]
 [-10   0   0   0  -8]
 [-18   0   0   0 -25]
 [-25 -39 -28 -41 -43]]

negative rewards -252 
positive rewards 160
